# Doing an exploration on recent disaster tweets for a disaster category

In [1]:
import pandas as pd
import pickle

import spacy
from spacy import displacy
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
import networkx as nx 
import nx_altair as nxa
from collections import Counter
import warnings

from tools import extract_assign_location, disaster_title, topNlocations, top_locations, \
                    sampleNtweets, top_locations_tweets, show_top_locations, get_top_locations_flatten, \
                    get_location_graph_data, build_network, get_layout, plot_disaster_mentions, is_intersect, \
                    show_tweets_locations, top_location_pairs

warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', 280)

## Define any upstream dependencies

In [2]:
upstream = [
    'vectorizer',
    'category_classification_models',
    'recent_tweets_wildfire',
    'recent_tweets_earthquake',
    'recent_tweets_flood',
    'recent_tweets_hurricane'
]
random_seed = None
disaster_kind = None

In [3]:
# Parameters
random_seed = 42
disaster_kind = "recent_tweets_earthquake"
upstream = {
    "vectorizer": {
        "nb": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/vectorizer.ipynb",
        "vectorizer": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/vectorizer.pkl",
        "vocab": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/vocab.pkl",
    },
    "category_classification_models": {
        "nb": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/category_classification_models.ipynb",
        "model_lr": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/model_lr.pkl",
        "model_rf": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/model_rf.pkl",
        "model_nb": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/model_nb.csv",
        "model_votingc": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/model_votingc.csv",
    },
    "recent_tweets_earthquake": {
        "nb": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/recent_tweets_earthquake.ipynb",
        "file": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/recent_tweets_earthquake.csv",
    },
    "recent_tweets_wildfire": {
        "nb": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/recent_tweets_wildfire.ipynb",
        "file": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/recent_tweets_wildfire.csv",
    },
    "recent_tweets_hurricane": {
        "nb": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/recent_tweets_hurricane.ipynb",
        "file": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/recent_tweets_hurricane.csv",
    },
    "recent_tweets_flood": {
        "nb": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/recent_tweets_flood.ipynb",
        "file": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/recent_tweets_flood.csv",
    },
}
product = {
    "nb": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/tweets_timeline_earthquake.ipynb"
}


In [4]:
print('Disaster kind: ', disaster_title(disaster_kind))

Disaster kind:  Earthquake


## Load the sample of recent tweets

In [5]:
df_tweets = pd.read_csv(upstream[disaster_kind]['file'])
df_tweets.sample(5)

,author_id,tweet_text,created_at,hashtags,users
74127,1112896007774371840,"1.90 magnitude earthquake occurred at Hawaii, HI, United States on 2022-08-03 20:55:23 UTC\nMap: https://t.co/tP77kWgart\n#earthb0t #earthquake #Hawaii #HI #UnitedStates #noaa #greatshakeout #LetsBeSafe #OpenData #data https://t.co/PuT9AQcBzo",2022-08-03 21:11:29+00:00,"['earthb0t', 'earthquake', 'Hawaii', 'HI', 'UnitedStates', 'noaa', 'greatshakeout', 'LetsBeSafe', 'OpenData', 'data']",[]
115626,1112896007774371840,"0.80 magnitude earthquake occurred at Matanuska-Susitna, AK, United States on 2022-08-01 08:56:01 UTC\nMap: https://t.co/XQOkub80pu\n#earthb0t #earthquake #MatanuskaSusitna #AK #UnitedStates #noaa #greatshakeout #LetsBeSafe #OpenData #data https://t.co/iqlKUsquQ9",2022-08-01 09:07:20+00:00,"['earthb0t', 'earthquake', 'MatanuskaSusitna', 'AK', 'UnitedStates', 'noaa', 'greatshakeout', 'LetsBeSafe', 'OpenData', 'data']",[]
36368,2157244796,RT @QuakesToday: 5.0 magnitude #earthquake. South Sandwich Islands region https://t.co/9vs6Odaevu,2022-08-05 07:01:40+00:00,['earthquake'],['QuakesToday']
4459,2780260122,RT @civildefensePH: The @NDRRMC_OpCen held an emergency full council meeting yesterday to discuss early recovery operations for the victims…,2022-08-06 12:41:13+00:00,[],"['civildefensePH', 'NDRRMC_OpCen']"
9094,1299383353263579136,"i feel the earthquake in the room and so i pray, fall into me and then—!",2022-08-05 18:18:35+00:00,[],[]


## Extract the locations from the tweet sample

In [6]:
%%time
df_tweets, df_locs_most_common = extract_assign_location(df_tweets, number_most_common=10)

df_tweets.sample(5)

0it [00:00, ?it/s]

12it [00:00, 116.59it/s]

43it [00:00, 227.13it/s]

73it [00:00, 257.22it/s]

105it [00:00, 277.76it/s]

...

143952it [06:46, 360.51it/s]

143989it [06:47, 359.44it/s]

144000it [06:47, 353.77it/s]

CPU times: user 8min 12s, sys: 5.35 s, total: 8min 18s
Wall time: 8min 16s


,author_id,tweet_text,created_at,hashtags,users,location
129200,1439672084842516481,"There was a 2.63 magnitude earthquake 5 km WNW of Jagual, Puerto Rico",2022-08-01 00:51:18+00:00,[],[],"[WNW, Jagual, Puerto Rico]"
40544,2409116011,"RT @grum10: #Earthquake: Southwest Of Sumatra, Indonesia at 04 August 2022 05:39 pm (UTC), Magnitude: 5.1 https://t.co/G0oN3BluGR",2022-08-04 23:07:35+00:00,['Earthquake'],['grum10'],[Indonesia]
27683,1554831633592725510,RT @OfficialDGISPR: CJCSC &amp; Services Chiefs express deepest condolences over tragic loss of precious lives &amp; damage to infrastructure due t…,2022-08-04 16:13:32+00:00,[],['OfficialDGISPR'],[]
45186,1501841604185038848,"RT @USCUPDiliman: Call for Donations\n\nLast week, Abra was hit by a 7.3 magnitude earthquake, and since then, the region has experienced con…",2022-08-04 09:48:40+00:00,[],['USCUPDiliman'],[]
37002,2170342764,RT @phivolcs_dost: #EarthquakePH #EarthquakeQuezon\nEarthquake Information No.1\nDate and Time: 05 August 2022 - 03:19 AM\nMagnitude = 1.4\nDep…,2022-08-04 19:28:17+00:00,"['EarthquakePH', 'EarthquakeQuezon']",['phivolcs_dost'],[]


In [7]:
df_locs_most_common

,location,count
0,United States,26630
1,Japan,5661
2,HI,5116
3,Hawaii,4186
4,Alaska,2463
5,Izu Islands,2387
6,CA,2201
7,Nepal,1817
8,Honaunau-Napoopoo,1779
9,Indonesia,1750


## Predict the class labels

In [8]:
# Get the vectorizer
with open(upstream["vectorizer"]["vectorizer"], 'rb') as f:
    vectorizer = pickle.load(f)

In [9]:
# Get the classification model
with open(upstream["category_classification_models"]["model_lr"], 'rb') as f:
    classifier = pickle.load(f)

In [10]:
%%time
# Predict the class labels
class_label = classifier.predict(vectorizer.transform(df_tweets['tweet_text']))
df_tweets['class_label'] = class_label

df_tweets.sample(5)

CPU times: user 31.3 s, sys: 104 ms, total: 31.4 s
Wall time: 31.4 s


,author_id,tweet_text,created_at,hashtags,users,location,class_label
116396,1112896007774371840,"1.51 magnitude earthquake occurred at San Benito, CA, United States on 2022-08-01 22:01:35 UTC\nMap: https://t.co/VdV3ia8P6X\n#earthb0t #earthquake #SanBenito #CA #UnitedStates #noaa #greatshakeout #LetsBeSafe #OpenData #data https://t.co/dHgcyjw8Rc",2022-08-01 22:06:49+00:00,"['earthb0t', 'earthquake', 'SanBenito', 'CA', 'UnitedStates', 'noaa', 'greatshakeout', 'LetsBeSafe', 'OpenData', 'data']",[],"[San Benito, United States]",other_relevant_information
85944,802064401,"#EarthquakePH #EarthquakeAbra\nEarthquake Information No.1\nDate and Time: 02 August 2022 - 08:25 PM\nMagnitude = 2.7\nDepth = 010 km\nLocation = 17.47°N, 120.71°E - 005 km N 05° E of Manabo (Abra)\n\nhttps://t.co/mYXScpwgC9 https://t.co/zcuCkUPf17",2022-08-02 12:37:30+00:00,"['EarthquakePH', 'EarthquakeAbra']",[],[Manabo],other_relevant_information
24981,1495126344434212864,RT @komonews: Did you feel the earthquake? \n\nhttps://t.co/3dgGJWZyIl,2022-08-05 05:20:50+00:00,[],['komonews'],[],not_humanitarian
136244,3278279106,RT @phivolcs_dost: #EarthquakePH #EarthquakeAbra\n#iFelt_AbraEarthquake\nEarthquake Information No.1\nDate and Time: 31 July 2022 - 09:45 PM\nM…,2022-08-01 06:20:20+00:00,"['EarthquakePH', 'EarthquakeAbra', 'iFelt_AbraEarthquake']",['phivolcs_dost'],[],other_relevant_information
131832,2170378830,"RT @gisliberg: A 5.08 magnitude earthquake has occurred near Iceland Region at 31.7.2022, 17:48! https://t.co/8hHsrb099I",2022-07-31 17:56:20+00:00,[],['gisliberg'],[Iceland Region],other_relevant_information


## Plot the intensity for each catagory, and for a particular location

### Prepare the time series grouped by disaster class label 

In [11]:
df_tweets['created_at'] = pd.to_datetime(df_tweets['created_at'], errors='coerce')
df_plot = df_tweets.copy()

df_plot['created_at'] = df_plot['created_at'].dt.floor('8H') # Round to 8 hours
df_plot['count'] = 1
df_group_plot = df_plot[['created_at', 'class_label', 'count']].groupby(['created_at', 'class_label']).count().reset_index()
df_group_plot['created_at'] = pd.to_datetime(df_group_plot['created_at'])
df_group_plot.head()

,created_at,class_label,count
0,2022-07-31 08:00:00+00:00,caution_and_advice,232
1,2022-07-31 08:00:00+00:00,infrastructure_and_utility_damage,153
2,2022-07-31 08:00:00+00:00,injured_or_dead_people,30
3,2022-07-31 08:00:00+00:00,not_humanitarian,13
4,2022-07-31 08:00:00+00:00,other_relevant_information,1923


### Prepare the time series grouped by location label 

In [12]:
df_explode = df_tweets.copy()

most_common10 = df_locs_most_common.head(10)['location'].tolist()

df_explode = df_explode.explode('location').reset_index(drop=True)
df_explode.dropna(inplace=True)
df_explode['created_at'] = df_explode['created_at'].dt.floor('8H') # Round to 8 hours
df_explode['count'] = 1
df_group_explode = df_explode[['created_at', 'location', 'count']].groupby(
    ['created_at', 'location']
).count().reset_index()
df_group_explode = df_group_explode[df_group_explode['location'].isin(most_common10)]
df_group_explode.head()

,created_at,location,count
7,2022-07-31 08:00:00+00:00,Alaska,2
29,2022-07-31 08:00:00+00:00,CA,21
70,2022-07-31 08:00:00+00:00,HI,25
71,2022-07-31 08:00:00+00:00,Hawaii,17
77,2022-07-31 08:00:00+00:00,Indonesia,2


### Find the top 10 locations in the tweets sample

In [13]:
top10_locs = list(df_group_explode['location'].unique())
top10_locs

['Alaska',
 'CA',
 'HI',
 'Hawaii',
 'Indonesia',
 'Izu Islands',
 'Japan',
 'Nepal',
 'United States',
 'Honaunau-Napoopoo']

In [14]:
chart = alt.Chart(df_group_explode[df_group_explode['location'].isin(top10_locs)]).mark_line(size=2).encode(
    x='created_at:T',
    y='count:Q',
    color='location:N',
    tooltip=['location', 'count']
)

chart.properties(
    width=500,
    height=500
).properties(
    title=disaster_title(disaster_kind) + ' disasters in the world, interactive chart'
).interactive()

alt.Chart(...)

## Combine the most important locations per category

In [15]:
df_top_locations = top_locations(df_tweets, top10_locs)
df_top_locations.sample(5)

,created_at,class_label,locations,count
116,2022-08-04 12:00:00+00:00,caution_and_advice,[United States],54
125,2022-08-04 20:00:00+00:00,caution_and_advice,"[Alaska, United States]",60
0,2022-07-31 10:00:00+00:00,caution_and_advice,[Nepal],1
145,2022-08-05 14:00:00+00:00,other_relevant_information,"[Alaska, HI, United States]",472
54,2022-08-02 02:00:00+00:00,infrastructure_and_utility_damage,[Indonesia],13


### Define the class labels we are interested in

In [16]:
class_labels = [
    'displaced_people_and_evacuations',
    'injured_or_dead_people',
    'rescue_volunteering_or_donation_effort',
    'requests_or_urgent_needs',
    'sympathy_and_support',
    'infrastructure_and_utility_damage',
    'caution_and_advice'
]

In [17]:
chart = alt.Chart(df_top_locations[df_top_locations['class_label'].isin(class_labels)]).mark_line(size=2).encode(
    x='created_at:T',
    y='count:Q',
    color='class_label:N',
    tooltip=['class_label:N', 'locations', 'count']
)

chart.properties(
    width=500,
    height=500
).properties(
    title=disaster_title(disaster_kind) + ' disasters in the world, interactive chart'
)

alt.Chart(...)

## Show sample tweets per most important locations above

In [18]:
df_top_location_tweets = top_locations_tweets(df_tweets, top10_locs)
df_top_location_tweets.sample(5)

,created_at,class_label,tweet_text,locations,count
103,2022-08-04 00:00:00+00:00,caution_and_advice,"[#Earthquake of magnitude 3.09 at 64 km N of Suárez, #Puerto_Rico. Retweet if you felt it., [🚨 EARLY WARNING] Earthquake Detected: A estimated magnitude 5.0 #地震 (#earthquake) with an estimated Shindo Intensity of 3 is occuring 福島県沖 (Off Fukushima Prefecture) with a depth of 4...","[Honaunau-Napoopoo, Japan, United States]",248
88,2022-08-03 04:00:00+00:00,other_relevant_information,"[RT @cnnphilippines: JUST IN: A magnitude 5.8 earthquake hits waters off Manay, Davao Oriental.\n\nPhivolcs says aftershocks are expected.\n\nFu…, 1.9 magnitude #earthquake. 26 km from #Honaunau-Napoopoo, HI, #UnitedStates https://t.co/zBdUiEwQ2W, RT @QuakesToday: 1.9 magnitu...","[HI, Honaunau-Napoopoo, United States]",2830
53,2022-08-02 02:00:00+00:00,caution_and_advice,"[#Earthquake of magnitude 3.05 at 20 km SE of Hana, #Hawaii. Retweet if you felt it., RT @earthquake247: #Earthquake of magnitude 4.1 at 289 km E of Levuka, #Fiji. Retweet if you felt it., #Earthquake of magnitude 2.95 at 20 km SE of Hana, #Hawaii. Retweet if you felt it., RT...",[United States],48
14,2022-07-31 22:00:00+00:00,other_relevant_information,"[Magnitude 6.0 Earthquake Reported in Nepal - teleSUR English https://t.co/T8axOosoRt, RT @phivolcs_dost: #EarthquakePH #EarthquakeAbra\nEarthquake Information No.1\nDate and Time: 01 August 2022 - 03:07 AM\nMagnitude = 2.3\nDepth…, 0.95 magnitude earthquake occurred at Seven...","[HI, Nepal, United States]",393
140,2022-08-05 10:00:00+00:00,other_relevant_information,"[#EarthquakePH #EarthquakeZambales\nEarthquake Information No.1\nDate and Time: 05 August 2022 - 07:03 PM\nMagnitude = 1.7\nDepth = 030 km\nLocation = 15.58°N, 120.11°E - 017 km N 72° E of Masinloc (Zambales)\n\nhttps://t.co/Qp0ESBP9n6 https://t.co/sBsrRsnh3w, 1.90 magnitude ...","[HI, Hawaii, United States]",1114


In [19]:
show_top_locations(df_top_location_tweets, class_labels, disaster_kind)

alt.VConcatChart(...)

## Visualize the locations and relationship between them as a network

### Select the top 20 locations for this disaster

In [20]:
loc20 = get_top_locations_flatten(df_tweets, 20)
loc20

['United States',
 'Japan',
 'HI',
 'Hawaii',
 'Alaska',
 'Izu Islands',
 'CA',
 'Nepal',
 'Honaunau-Napoopoo',
 'Indonesia',
 'Abra',
 'Ilocos Sur',
 'New',
 'Kudarat',
 'Lebak',
 'Luzon',
 'Balochistan',
 'Afghanistan',
 'Honduras',
 'Iceland']

### Get the tweets for just the top 20 locations

In [21]:
df_loc_graph = get_location_graph_data(df_tweets, loc20)
df_loc_graph.sample(10)

,class_label,location_x,location_y,count
3,caution_and_advice,Japan,Izu Islands,128
9,infrastructure_and_utility_damage,Luzon,Ilocos Sur,25
27,other_relevant_information,United States,Honaunau-Napoopoo,1234
21,other_relevant_information,New,Balochistan,7
18,other_relevant_information,Japan,Izu Islands,1796
22,other_relevant_information,New,Japan,377
10,injured_or_dead_people,Luzon,Abra,8
20,other_relevant_information,Luzon,Abra,44
12,other_relevant_information,Honaunau-Napoopoo,HI,709
4,caution_and_advice,United States,Alaska,298


In [22]:
plot_disaster_mentions(df_loc_graph, 'caution_and_advice', 'Caution And Advice')

alt.LayerChart(...)

### Get the most frequent location pair

In [23]:
loc_pairs = top_location_pairs(df_loc_graph, 'caution_and_advice', top_pairs_count=1)
loc_pairs

[['United States', 'Alaska']]

### Show a sample for this location pair

In [24]:
show_tweets_locations(df_tweets, class_label='caution_and_advice' , locations=loc_pairs[0], sample=5)

,tweet_text,location
0,"RT @EQAlerts: Moderate magnitude 4.4 #earthquake 21 miles northeast of Dutch Harbor, Alaska, United States - info, user reports and updates…","[Dutch Harbor, Alaska, United States]"
1,"RT @EQAlerts: Significant magnitude 5.1 #earthquake 50 miles southeast of Nikolski, Alaska, United States - info, user reports and updates…","[Nikolski, Alaska, United States]"
2,"RT @EQAlerts: Moderate magnitude 4.4 #earthquake 21 miles northeast of Dutch Harbor, Alaska, United States - info, user reports and updates…","[Dutch Harbor, Alaska, United States]"
3,"RT @EQAlerts: Significant magnitude 5.1 #earthquake 50 miles southeast of Nikolski, Alaska, United States - info, user reports and updates…","[Nikolski, Alaska, United States]"
4,"1.9 magnitude #earthquake. 32 km from #Honaunau-Napoopoo, HI, United States https://t.co/M7Ax2Z4Wmt\n1.8 magnitude #earthquake. 9 km from Truckee, #CA, United States https://t.co/iENGLgtO6N\n2.9 magnitude #earthquake. 43 km SW of Skwentna, Alaska https://t.co/12AkxeQS9Z\n.......","[Honaunau-Napoopoo, HI, United States, Truckee, United States, Skwentna, Alaska]"


In [25]:
plot_disaster_mentions(df_loc_graph, 'displaced_people_and_evacuations', 'Displaced People And Evacuations')

'No records found for class displaced_people_and_evacuations'

### Get the most frequent location pair

In [26]:
loc_pairs = top_location_pairs(df_loc_graph, 'displaced_people_and_evacuations', top_pairs_count=1)
loc_pairs

[[]]

### Show a sample for this location pair

In [27]:
show_tweets_locations(df_tweets, class_label='displaced_people_and_evacuations' , locations=loc_pairs[0], sample=5)

'No location pairs provided'

In [28]:
plot_disaster_mentions(df_loc_graph, 'injured_or_dead_people', 'Injured Or Dead People')

alt.LayerChart(...)

### Get the most frequent location pair

In [29]:
loc_pairs = top_location_pairs(df_loc_graph, 'injured_or_dead_people', top_pairs_count=1)
loc_pairs

[['Luzon', 'Abra']]

### Show a sample for this location pair

In [30]:
show_tweets_locations(df_tweets, class_label='injured_or_dead_people' , locations=loc_pairs[0], sample=5)

,tweet_text,location
0,"The 7-magnitude earthquake hit the northern #Philippine provinces of Abra, Luzon, and Metropolitan Manila on July 27. The NDRRMC report says a total of 314,161 people have been affected by the earthquake, with a total of 10 people killed, and 375 others wounded. https://t.co/...","[Philippine, Abra, Luzon]"
1,"The 7-magnitude earthquake hit the northern #Philippine provinces of Abra, Luzon, and Metropolitan Manila on July 27. The NDRRMC report says a total of 314,161 people have been affected by the earthquake, with a total of 10 people killed, and 375 others wounded. https://t.co/...","[Philippine, Abra, Luzon]"
2,"The 7-magnitude earthquake hit the northern #Philippine provinces of Abra, Luzon, and Metropolitan Manila on July 27. The NDRRMC report says a total of 314,161 people have been affected by the earthquake, with a total of 10 people killed, and 375 others wounded. https://t.co/...","[Philippine, Abra, Luzon]"
3,"The 7-magnitude earthquake hit the northern #Philippine provinces of Abra, Luzon, and Metropolitan Manila on July 27. The NDRRMC report says a total of 314,161 people have been affected by the earthquake, with a total of 10 people killed, and 375 others wounded. https://t.co/...","[Philippine, Abra, Luzon]"
4,"The 7-magnitude earthquake hit the northern #Philippine provinces of Abra, Luzon, and Metropolitan Manila on July 27. The NDRRMC report says a total of 314,161 people have been affected by the earthquake, with a total of 10 people killed, and 375 others wounded. https://t.co/...","[Philippine, Abra, Luzon]"


In [31]:
plot_disaster_mentions(df_loc_graph, 'rescue_volunteering_or_donation_effort',
                       'Rescue Volunteering Or Donation Effort')

alt.LayerChart(...)

### Get the most frequent location pair

In [32]:
loc_pairs = top_location_pairs(df_loc_graph, 'rescue_volunteering_or_donation_effort', top_pairs_count=1)
loc_pairs

[['Ilocos Sur', 'Abra']]

### Show a sample for this location pair

In [33]:
show_tweets_locations(df_tweets, class_label='rescue_volunteering_or_donation_effort',
                      locations=loc_pairs[0], sample=5)

,tweet_text,location
0,"RT @manilabulletin: The Philippine Red Cross (PRC) has distributed 201,427 liters of water to 2,212 families in Abra and Ilocos Sur to aid…","[Abra, Ilocos Sur]"
1,"The Philippine Red Cross (PRC) has distributed 201,427 liters of water to 2,212 families in Abra and Ilocos Sur to aid the communities’ need for adequate water supply after a magnitude 7.0 earthquake hit Abra on July 27.\n\nhttps://t.co/RO8BA54LZX","[Philippine Red Cross, Abra, Ilocos Sur]"
2,"RT @manilabulletin: The Philippine Red Cross (PRC) has distributed 201,427 liters of water to 2,212 families in Abra and Ilocos Sur to aid…","[Abra, Ilocos Sur]"
3,"RT @manilabulletin: The Philippine Red Cross (PRC) has distributed 201,427 liters of water to 2,212 families in Abra and Ilocos Sur to aid…","[Abra, Ilocos Sur]"
4,"The Philippine Red Cross (PRC) has distributed 201,427 liters of water to 2,212 families in Abra and Ilocos Sur to aid the communities’ need for adequate water supply after a magnitude 7.0 earthquake hit Abra on July 27.\nhttps://t.co/XWpxClbDKA","[Philippine Red Cross, Abra, Ilocos Sur]"


In [34]:
plot_disaster_mentions(df_loc_graph, 'requests_or_urgent_needs',
                       'requests_or_urgent_needs')

'No records found for class requests_or_urgent_needs'

### Get the most frequent location pair

In [35]:
loc_pairs = top_location_pairs(df_loc_graph, 'requests_or_urgent_needs', top_pairs_count=1)
loc_pairs

[[]]

### Show a sample for this location pair

In [36]:
show_tweets_locations(df_tweets, class_label='requests_or_urgent_needs',
                      locations=loc_pairs[0], sample=5)

'No location pairs provided'

In [37]:
plot_disaster_mentions(df_loc_graph, 'sympathy_and_support',
                       'sympathy_and_support')

alt.LayerChart(...)

### Get the most frequent location pair

In [38]:
loc_pairs = top_location_pairs(df_loc_graph, 'sympathy_and_support', top_pairs_count=1)
loc_pairs

[['Nepal', 'Balochistan']]

### Show a sample for this location pair

In [39]:
show_tweets_locations(df_tweets, class_label='sympathy_and_support',
                      locations=loc_pairs[0], sample=5)

,tweet_text,location
0,5.6 Magnitude earthquake jolts coastal parts of Balochistan...!! \nMay Allah protect our country! Ameen..\n#earthquake #Balochistan #Nepal https://t.co/Sgv0JaAwaZ,"[Balochistan, Nepal]"
1,5.6 Magnitude earthquake jolts coastal parts of Balochistan...!! \nMay Allah protect our country! Ameen..\n#earthquake #Balochistan #Nepal https://t.co/Sgv0JaAwaZ,"[Balochistan, Nepal]"
2,5.6 Magnitude earthquake jolts coastal parts of Balochistan...!! \nMay Allah protect our country! Ameen..\n#earthquake #Balochistan #Nepal https://t.co/Sgv0JaAwaZ,"[Balochistan, Nepal]"
3,5.6 Magnitude earthquake jolts coastal parts of Balochistan...!! \nMay Allah protect our country! Ameen..\n#earthquake #Balochistan #Nepal https://t.co/Sgv0JaAwaZ,"[Balochistan, Nepal]"
4,5.6 Magnitude earthquake jolts coastal parts of Balochistan...!! \nMay Allah protect our country! Ameen..\n#earthquake #Balochistan #Nepal https://t.co/Sgv0JaAwaZ,"[Balochistan, Nepal]"


In [40]:
plot_disaster_mentions(df_loc_graph, 'infrastructure_and_utility_damage',
                       'infrastructure_and_utility_damage')

alt.LayerChart(...)

### Get the most frequent location pair

In [41]:
loc_pairs = top_location_pairs(df_loc_graph, 'infrastructure_and_utility_damage', top_pairs_count=1)
loc_pairs

[['Ilocos Sur', 'Abra']]

### Show a sample for this location pair

In [42]:
show_tweets_locations(df_tweets, class_label='infrastructure_and_utility_damage',
                      locations=loc_pairs[0], sample=5)

,tweet_text,location
0,"A total of 20,533 houses in northern Luzon were heavily damaged by the magnitude 7 earthquake that jolted the provinces of Abra, Ilocos Sur, La Union, Pangasinan, Kalinga Apayao, Benguet, Ifugao, and Mountain Province on July 27. https://t.co/LF19bOztui","[Luzon, Abra, Ilocos Sur, La Union, Pangasinan, Kalinga Apayao, Benguet, Ifugao, Mountain Province]"
1,"A total of 20,533 houses in northern Luzon were heavily damaged by the magnitude 7 earthquake that jolted the provinces of Abra, Ilocos Sur, La Union, Pangasinan, Kalinga Apayao, Benguet, Ifugao, and Mountain Province on July 27. https://t.co/LF19bOztui","[Luzon, Abra, Ilocos Sur, La Union, Pangasinan, Kalinga Apayao, Benguet, Ifugao, Mountain Province]"
2,"A team consisting of seven faculty and staff members from ICE has just concluded their post-disaster damage assessment in select heritage sites in Vigan, Ilocos Sur following the recent earthquake that occured in Abra last July 27. #ServeThePeople #DRRM #HeritageSites https:/...","[Vigan, Ilocos Sur, Abra]"
3,"ICYMI: A total of 20,533 houses in northern Luzon were heavily damaged by the magnitude 7 earthquake that jolted the provinces of Abra, Ilocos Sur, La Union, Pangasinan, Kalinga Apayao, Benguet, Ifugao, and Mountain Province on July 27. https://t.co/LF19bOztui","[Luzon, Abra, Ilocos Sur, La Union, Pangasinan, Kalinga Apayao, Benguet, Ifugao, Mountain Province]"
4,"A 5.1 magnitude earthquake hit Villaviciosa, Abra at 2:48am. Phivolcs says damage can be expected. \n\nIntensity 5 shaking was felt in parts of Abra, Vigan City, Ilocos Sur, Ilocos Norte, and Mountain Province. Lower intensities were also felt in other parts of Northern Luzo...","[Villaviciosa, Abra, Abra, Vigan City, Ilocos Sur, Ilocos Norte, Mountain Province, Northern Luzon]"
